![Microsoft](https://raw.githubusercontent.com/microsoft/azuredatastudio/master/src/sql/media/microsoft-small-logo.png)

## Create Azure Kubernetes Service cluster and deploy SQL Server 2019 big data cluster

This notebook walks through the process of deploying a SQL Server 2019 big data cluster on Azure Kubernetes Service.

* Please follow the instructions in the **Dependencies** cell to install the dependencies.
* The **Required information** cell will prompt you for the required information to create a SQL Server 2019 big data cluster.
* We have included some default settings in the **Azure settings** cell of this Notebook, Please review the default values and make changes accordingly.
* If you are experiencing issue in the *Create AKS cluster* step you might need to re-run the cell, more information: https://docs.microsoft.com/en-us/azure/aks/kubernetes-service-principal#specify-a-service-principal-for-an-aks-cluster. 



### **Dependencies**
<table>
<colgroup>
<col style="width: 10%" />
<col style="width: 80%" />
<col style="width: 10%" />
</colgroup>
<thead>
<tr class="header">
<th>Tool</th>
<th>Description</th>
<th>Installation</th>
</tr>
</thead>
<tbody>
<tr>
<td><strong>Azure CLI</strong></td>
<td>Command-line interface for managing Azure services. Used with AKS big data cluster deployments (<a href="https://docs.microsoft.com/cli/azure/?view=azure-cli-latest">More info</a>).</td>
<td><a href="https://docs.microsoft.com/cli/azure/install-azure-cli?view=azure-cli-latest">Install</a></td>
</tr>
<tr>
<td><strong>kubectl</strong></td>
<td>Command-line tool for monitoring the underlying Kuberentes cluster (<a href="https://kubernetes.io/docs/tasks/tools/install-kubectl/">More info</a>).</td>
<td><a href="https://kubernetes.io/docs/tasks/tools/install-kubectl/#install-kubectl-binary-using-native-package-management">Install</a></td>
</tr>
<tr>
<td><strong>mssqlctl</strong></td>
<td>Command-line tool for installing and managing a big data cluster.</td>
<td><a href="https://docs.microsoft.com/en-us/sql/big-data-cluster/deploy-install-mssqlctl?view=sqlallproducts-allversions">Install</a></td>
</tr>
</tbody>
</table>
<p>

### **Check dependencies**

In [1]:
#Run command helper function
def run_command():
    print("Executing: " + cmd)
    !{cmd}
    if _exit_code != 0:
        raise SystemExit(f'Shell command:\n\n\t{cmd}\n\nreturned non-zero exit code: ' + str(_exit_code) + '.\n')
    print(f'Successfully executed: {cmd}')

cmd = 'az --version'
run_command()
cmd = 'kubectl version --client=true'
run_command()
cmd = 'mssqlctl --version'
run_command()

### **Required information**

In [2]:
import getpass
def get_user_input(input_name, is_password = False, confirm_password = False):
    if is_password:
        user_input = getpass.getpass(prompt = input_name)
        if confirm_password:
            user_input_confirm = getpass.getpass(prompt = 'Confirm '+ input_name)
            if user_input != user_input_confirm:
                raise SystemExit(f'{input_name} does not match the confirmation password')
        print(f'{input_name}: ******')
    else:
        user_input = input(input_name)
        print(input_name + ': ' + user_input)
    if user_input == "":
        raise SystemExit(f'{input_name} is required')
    return user_input;

mssql_cluster_name = get_user_input('Cluster name')
bdc_controller_username = get_user_input('Controller username')
bdc_controller_password = get_user_input('Controller password', True, True)
bdc_mssql_sa_password = get_user_input('SQL Server password', True, True)
docker_username = get_user_input('Docker username')
docker_password = get_user_input('Docker password', True)

### **Azure settings**
*Subscription ID*: visit <a href="https://portal.azure.com/#blade/Microsoft_Azure_Billing/SubscriptionsBlade">here</a> to find out the subscriptions you can use, if you leave it unspecified, the default subscription will be used.

*VM Size*: visit <a href="https://docs.microsoft.com/en-us/azure/virtual-machines/linux/sizes">here</a> to find out the available VM sizes you could use. 
 
*Region*: visit <a href="https://azure.microsoft.com/en-us/global-infrastructure/services/?products=kubernetes-service">here</a> to find out the Azure regions where the Azure Kubernettes Service is available.

In [3]:
azure_subscription_id = ""
azure_vm_size = "Standard_E4s_v3"
azure_region = "eastus"
azure_vm_count = int(5)

### **Default settings**

In [4]:
azure_resource_group = mssql_cluster_name
bdc_knox_password = bdc_mssql_sa_password
aks_kubernetes_version = ''
aks_cluster_name = mssql_cluster_name
mssqlctl_configuration_profile = 'aks-dev-test'
mssqlctl_configuration_file = 'mssql-bdc-configuration'

# Show parameter values
print('')
print(f'mssql_cluster_name = {mssql_cluster_name}')
print(f'azure_subscription_id = {azure_subscription_id}')
print(f'azure_vm_size = {azure_vm_size}')
print(f'azure_vm_count = {str(azure_vm_count)}')
print(f'azure_region = {azure_region}')
print(f'azure_resource_group = {azure_resource_group}')
print(f'aks_kubernetes_version = {aks_kubernetes_version}')
print(f'aks_cluster_name = {aks_cluster_name}')
print(f'docker_username = {docker_username}')
print(f'mssqlctl_configuration_profile = {mssqlctl_configuration_profile}')
print(f'mssqlctl_configuration_file = {mssqlctl_configuration_file}')
print(f'bdc_controller_username = {bdc_controller_username}')
print('')

### **Login to Azure**

This will open a web browser window to enable credentials to be entered. If this cells is hanging forever, it might be because your Web browser windows is waiting for you to enter your Azure credentials!


In [5]:
cmd = f'az login'
run_command()


### **Set active Azure subscription**

In [6]:
if azure_subscription_id != "":
    cmd = f'az account set --subscription {azure_subscription_id}'
    run_command()
else:
    print('Using the default Azure subscription', {azure_subscription_id})
cmd = f'az account show'
run_command()

### **Create Azure resource group**

In [7]:
cmd = f'az group create --name {azure_resource_group} --location {azure_region}'
run_command()

### **Create AKS cluster**

In [8]:
if aks_kubernetes_version != "":
    kubernetes_version_option="--kubernetes-version {aks_kubernetes_version}"
else:
    kubernetes_version_option = ""
cmd = f'az aks create --name {aks_cluster_name} --resource-group {azure_resource_group} --generate-ssh-keys --node-vm-size {azure_vm_size} --node-count {azure_vm_count} {kubernetes_version_option}' 
run_command()

### **Set the new AKS cluster as current context**

In [9]:
cmd = f'az aks get-credentials --resource-group {azure_resource_group} --name {aks_cluster_name} --admin --overwrite-existing'
run_command()

### **List the MSSQLCTL configuration profiles**

In [10]:
import os
os.environ["ACCEPT_EULA"] = 'yes'
cmd = f'mssqlctl bdc config list'
run_command()

### **Create a MSSQLCTL configuration file**

In [11]:
# Create a configuration file
cmd = f'mssqlctl bdc config init --source {mssqlctl_configuration_profile} --target {mssqlctl_configuration_file} --force'
run_command()

# Set the 'big data cluster' name
jsonPath = '"metadata.name=''{0}''"'.format(mssql_cluster_name)
cmd = f'mssqlctl bdc config section set -c {mssqlctl_configuration_file} -j {jsonPath}'
run_command()

### **Create SQL Server 2019 big data cluster**

In [12]:
import os
print (f'Creating SQL Server 2019 big data cluster: {mssql_cluster_name} using configuration file {mssqlctl_configuration_file}')
os.environ["CONTROLLER_USERNAME"] = bdc_controller_username
os.environ["CONTROLLER_PASSWORD"] = bdc_controller_password
os.environ["MSSQL_SA_PASSWORD"] = bdc_mssql_sa_password
os.environ["KNOX_PASSWORD"] = bdc_knox_password
os.environ["DOCKER_USERNAME"] = docker_username
os.environ["DOCKER_PASSWORD"] = docker_password

cmd = f'mssqlctl bdc create -c {mssqlctl_configuration_file} --accept-eula yes'
run_command()

### **Login to SQL Server 2019 big data cluster**

In [13]:
cmd = f'mssqlctl login --cluster-name {mssql_cluster_name}'
run_command()

### **Show SQL Server 2019 big data cluster endpoints**

In [14]:
import json
import pandas as pd
from IPython.display import *
pd.set_option('display.max_colwidth', -1)

def formatColumnNames(column):
    return ' '.join(word[0].upper() + word[1:] for word in column.split())

def show_results(results):
    df = pd.DataFrame(results)
    df.columns = [formatColumnNames(n) for n in results[0].keys()]
    mydata = HTML(df.to_html(render_links=True))
    display(mydata)

cmd = f'mssqlctl bdc endpoint list'
endpointsResults  = !{cmd}
endpointsInJson = json.loads(''.join(endpointsResults))
show_results(endpointsInJson)

### **Connect to master SQL Server instance in Azure Data Studio**
Click the link below to connect to the master SQL Server instance of the SQL Server 2019 big data cluster.

In [15]:
filteredEndpoints = [x for x in endpointsInJson if x['name'] == 'sql-server-master']
if filteredEndpoints and len(filteredEndpoints) == 1:
    display(HTML("<h3><a href=\"command:azdata.connect?{&quot;serverName&quot;:&quot;"+filteredEndpoints[0]['endpoint']+"&quot;,&quot;providerName&quot;:&quot;MSSQL&quot;, &quot;authenticationType&quot;:&quot;SqlLogin&quot;,&quot;userName&quot;:&quot;sa&quot;,&quot;password&quot;:&quot;"+bdc_mssql_sa_password+"&quot;}\">Click here to connect to SQL Server</a></h3>"))